In [1]:
from datasets import load_dataset

c:\Users\USER\anaconda3\envs\maizeDisease\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset  =  load_dataset("aquib1011/maize-leaf-disease")

In [3]:
import os
# Create directories for images
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/test', exist_ok=True)

In [4]:
!pip install pillow

In [5]:
!pip install datasets pillow tensorflow

In [6]:
# Save images from dataset
def save_images(dataset_split, split_name):
    for i, item in enumerate(dataset_split):
        image = item['image']
        
        # Convert RGBA image to RGB if needed
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        
        image.save(f'data/{split_name}/image_{i}.jpg')

# Save images to directories
save_images(dataset['train'], 'train')
save_images(dataset['test'], 'test')

In [7]:
!pip install tensorflow

In [8]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# test_datagen = ImageDataGenerator(rescale=1./255)

# # Load images using flow_from_directory with subdirectories
# train_generator = train_datagen.flow_from_directory(
#     'data/train',  # Path to the directory containing training images
#     target_size=(150, 150),  # Resize images to (150, 150)
#     batch_size=32,
#     class_mode=None,  # No class mode since there are no classes
#     shuffle=True  # Shuffle the data for training
# )

# test_generator = test_datagen.flow_from_directory(
#     'data/test',  # Path to the directory containing test images
#     target_size=(150, 150),  # Resize images to (150, 150)
#     batch_size=32,
#     class_mode=None,  # No class mode since there are no classes
#     shuffle=False  # No need to shuffle test data
# )


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 3350 images belonging to 1 classes.
Found 838 images belonging to 1 classes.


In [9]:
from PIL import Image
import os

# Example to check the first image in 'data/train'
image_path = './data/train/image_0.jpg'
try:
    img = Image.open(image_path)
    img.show()  # Opens the image using the default viewer
except Exception as e:
    print(f"Error opening image: {e}")


In [10]:
!pip install scipy

In [11]:
import scipy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use 'softmax' and adjust Dense units for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2
)


Epoch 1/2
105/105 [==============================] - 126s 1s/step - loss: 1.6587e-38 - accuracy: 1.0000
Epoch 2/2
105/105 [==============================] - 130s 1s/step - loss: 1.4250e-38 - accuracy: 1.0000


In [14]:
model.save('maize_leaf_disease_model.h5')


c:\Users\USER\anaconda3\envs\maizeDisease\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
!pip install pandas